In [2]:
import pandas as pd
import numpy as np
import evaluate
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer
from statistics import mean 
from datasets import Dataset
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import os
import re

path = '1/preds1' # '3/preds3'
files = os.listdir(path)

chrf = evaluate.load("chrf")
 
def cosine_similarity(a, b):
    return np.dot(a, b)/(norm(a)*norm(b))

In [3]:
dataf1 = "../text generation/datafiles/generated output/merge_broca.csv"
data = pd.read_csv(dataf1)#["preprocessed_text"]
data = Dataset.from_pandas(data).train_test_split(test_size=0.3, seed=42)
prompts = list(pd.read_csv("1/modified_test1.csv")['prompt'])# list(data['test']['modified'])

all_answers = "1/eval_answer1.csv" # "3/eval_answer3.csv"
answers = list(pd.read_csv(all_answers)["answer"])

In [4]:
print(len(prompts), len(answers))

3477 3477


In [5]:
def atoi(text):
    return int(text) if text.isdigit() else text

In [6]:
def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [7]:
files.sort(key=natural_keys)

In [8]:
files

['preds200.npy',
 'preds400.npy',
 'preds600.npy',
 'preds800.npy',
 'preds1000.npy',
 'preds1200.npy',
 'preds1400.npy',
 'preds1600.npy',
 'preds1800.npy',
 'preds2000.npy',
 'preds2200.npy',
 'preds2400.npy',
 'preds2600.npy',
 'preds2800.npy',
 'preds3000.npy',
 'preds3200.npy',
 'preds3400.npy',
 'preds3477.npy']

In [11]:
prev_start = 0
len_r = 0
for file in files:
    results = path+'/'+file
    num = re.search('\d+', file).group()
    save = "1/results1/score_results"+num # "3/results3/score_results"+num
    results = np.load(results)
    results = results.tolist()
    start = prev_start
    end = int(num)  
    print(f"Start: {start}, End: {end}")
    len_r += len(results)
    curr_prompts = prompts[start:end]
    curr_answers = answers[start:end]
    
    max_cosine = []
    max_chrf = []
    
    max_cosine_sents = []
    max_chrf_sents = []
    # true_labels = []
    
    for i in range(len(results)):
        answer = curr_answers[i]
        preds = results[i]
        chrfs = []
        cosine = []
        
        if i % 10 == 0:
            print(f"File:{file}, example {i}")
            
        for pred in preds:
            # print(pred)
            # print(answer)
            # print('\n')
            chrfs.append(chrf.compute(predictions=[pred], references=[answer])["score"])
            embeds_preds = model.encode(pred)
            embeds_true = model.encode(answer)
            cosine.append(cosine_similarity(embeds_true, embeds_preds))

        max_chrf.append(max(chrfs))
        max_cosine.append(max(cosine))

        # true_labels.append(answer)
        max_chrf_sents.append(preds[chrfs.index(max(chrfs))])
        max_cosine_sents.append(preds[cosine.index(max(cosine))])
        # break

    pd.DataFrame({"input": curr_prompts, "true": curr_answers, "max_chrf_pred": max_chrf_sents, "max_cosine_pred": max_cosine_sents, "max_chrf":max_chrf , "max_cosine": max_cosine}).to_csv(save+".csv", index=False)

    f = open(save+".txt", "w")
    str = (f"Average of max value from 5 generations\n"
           f"chrf: {mean(max_chrf)}\n"
           f"semantic similarity: {mean(max_cosine)}\n")
    f.write(str)
    f.close()
    prev_start = int(num)
print(len_r)

Start: 0, End: 200
File:preds200.npy, example 0
File:preds200.npy, example 10
File:preds200.npy, example 20
File:preds200.npy, example 30
File:preds200.npy, example 40
File:preds200.npy, example 50
File:preds200.npy, example 60
File:preds200.npy, example 70
File:preds200.npy, example 80
File:preds200.npy, example 90
File:preds200.npy, example 100
File:preds200.npy, example 110
File:preds200.npy, example 120
File:preds200.npy, example 130
File:preds200.npy, example 140
File:preds200.npy, example 150
File:preds200.npy, example 160
File:preds200.npy, example 170
File:preds200.npy, example 180
File:preds200.npy, example 190
Start: 200, End: 400
File:preds400.npy, example 0
File:preds400.npy, example 10
File:preds400.npy, example 20
File:preds400.npy, example 30
File:preds400.npy, example 40
File:preds400.npy, example 50
File:preds400.npy, example 60
File:preds400.npy, example 70
File:preds400.npy, example 80
File:preds400.npy, example 90
File:preds400.npy, example 100
File:preds400.npy, ex

In [ ]:
# max_cosine = []
# max_chrf = []
# 
# max_cosine_sents = []
# max_chrf_sents = []
# true_labels = []
# 
# for i in range(len(results)):
#     answer = answers[i]
#     preds = results[i]
#     
#     chrfs = []
#     cosine = []
#     
#     print(i)
#         
#     for pred in preds:
#         
#         chrfs.append(chrf.compute(predictions=[pred], references=[answer])["score"])
#         embeds_preds = model.encode(pred)
#         embeds_true = model.encode(answer)
#         cosine.append(cosine_similarity(embeds_true, embeds_preds))
#         
#         # print(pred)
#         # print(answer)
#         # print()
#         # sc1 = []
#         # sc2 = []
#         # sents = sent_tokenize(pred)
#         # print(pred)
#         # print(answer)
#         # print()
#         # true_sents = sent_tokenize(answer)
#         # for s, t in zip(sents, true_sents):
#         #     # print(s)
#         #     # print(t)
#         #     # print() 
#         #     sc1.append(chrf.compute(predictions=[s], references=[t])["score"])
#         # 
#         #     embeds_preds = model.encode(pred)
#         #     embeds_true = model.encode(answer)
#         # 
#         #     sc2.append(cosine_similarity(embeds_true, embeds_preds))
#         # chrfs.append(mean(sc1))
#         # cosine.append(mean(sc2))
#         
#     max_chrf.append(max(chrfs))
#     max_cosine.append(max(cosine))
#     
#     true_labels.append(answer)
#     max_chrf_sents.append(preds[chrfs.index(max(chrfs))])
#     max_cosine_sents.append(preds[cosine.index(max(cosine))])